In [1]:
import torch 
####
#TODO

# aussi il y a un biais au debut quil faudrait corriger en fonction de la loneur du path
# peut etre qu il faudrait calculer le nb de possibile paths
# reduire le sampling 

# faire symbolic
# warnings
# initialiser h0 pour rontrainer
# check if sub and divide takes x0 and x1 in the good order

import numpy as np
from Loss import Loss
from RON import RecurrentOccamNet
from RONTrainer import RONTrainer
from cocob import COCOBBackprop 
from bases import *

X = [[np.array([np.cos(i), 1/100], dtype=np.float64)] for i in range(5)]

Y = []
W = []
w = np.array([1., 1.])
for x in X:
    y = np.sin(np.dot(x[0],w))
    w += x[0]*y
    Y.append([y])
    W.append(np.array(w.copy()))


In [2]:
r_model = RecurrentOccamNet([2], 2, [1], 
                            [[on_sin(), on_cos(), on_sum(), on_dot(), on_mul()],
                             [on_sin(), on_cos(), on_sum(), on_dot(), on_mul()],
                             [on_sin(), on_cos(), on_sum(), on_dot(), on_mul()], 
                            [on_identity(), on_identity()]], 
                            [0.01, 0.01, 0.01, 0.01], 
                            skip_connections=True, is_none=False)
criterion = Loss(0.01)
optimizer = COCOBBackprop(r_model.parameters())
trainer = RONTrainer(r_model, criterion, optimizer)

trained_model = trainer.fit(X, Y, 4, 3)

epoch 0
[(array([0.54030231, 0.99995   ]), array([0.85755322, 0.99995   ]), array([0.91465333, 0.99995   ]), array([0.54869613, 0.99995   ]), array([0.79387345, 0.99995   ])), (array([1.   , 0.505]), array([5.14687546, 0.53529983]), array([8.17438951, 0.56741764]), array([-0.20639517,  0.60146251]), array([33.0877184 ,  0.63755006])), (array([0.21939564, 0.21939564]), array([0.54780817, 0.35683042]), array([-0.02999662, -1.34182332]), array([-2.77744227, -5.27460726]), array([ -7.27241243, -11.10256302]))]
[(array([0.750025, 0.750025]), array([0.99644707, 0.99644707]), array([1.33932268, 1.33932268]), array([6.04902854, 6.04902854]), array([195.22701049, 195.22701049])), (array([3.34147098, 0.52999983]), array([14.00076256,  0.56179965]), array([-20.62404245,   0.59550744]), array([95.53084739,  0.63123769]), array([-270.42303898,    0.66911174])), (array([1.0424085 , 1.38318923]), array([1.58277053, 1.93666109]), array([2.18904561, 2.53434232]), array([2.78551954, 3.38033564]), array(

RuntimeError: The size of tensor a (3) must match the size of tensor b (5) at non-singleton dimension 0

In [ ]:
list(trained_model.parameters())

[Parameter containing:
 tensor([[1.0000, 1.0000],
         [1.0000, 1.0000],
         [0.9900, 1.0100],
         [0.9900, 1.0100],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000]], dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor([[1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.]], dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
 

In [ ]:
trained_model.samples_paths_and_probas(3)


([[[<bases.on_identity at 0x1683710c0>,
    0],
   [<bases.on_identity at 0x168371060>, <bases.on_sin at 0x10dcd7d00>, 0]],
  [[<bases.on_identity at 0x1683710c0>, <bases.on_sin at 0x168373880>, 1],
    1,
    1]],
  [[<bases.on_identity at 0x1683710c0>,
    1,
    1,
    1,
    1],
    0,
    0]]],
 [[[4, [0]], [4, [1]], [2, [1]], [1, [1]], [1, [0]]],
  [[4, [0]], [4, [1]], [3, [0]], [3, [4, 5]], [1, [1]]],
  [[4, [0]],
   [4, [1]],
   [3, [4, 5]],
   [2, [6, 7]],
   [1, [0]],
   [2, [6, 7]],
   [1, [0]],
   [1, [1]],
   [2, [2, 3]],
   [1, [0]]]],
 [[[8], [2], [3], [0], [0]],
  [[12], [15], [1], [1, 3], [1]],
  [[15], [9], [11, 11], [2, 1], [1], [2, 3], [1], [1], [2, 0], [0]]])

In [ ]:
import copy
nb_sampled_paths = 1

#X = [x.append(np.array([None])) for x in X]
X = [[copy.deepcopy(x) for _ in range(nb_sampled_paths)] for x in X]
# Reshape Y so that shape == (nb elements, output dim).
Y = torch.tensor(Y, requires_grad = False)

the_paths, the_keys_probas, the_keys_path2 = trained_model.samples_paths_and_probas(nb_sampled_paths)

for x, y in zip(X, Y):
    # This imply to compute the pred of the model for each sampled path
    pred = trained_model.forward(x, the_paths) # shape == (nb sampled paths, output dim)
    print(pred, y)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [ ]:
print(trained_model.forward(X, the_path)[1])
print(W[:n])

[array([[1.5       , 1.04030231]]), array([[2.04030231, 2.04030231]]), array([[1.62415547, 2.58060461]]), array([[0.63416297, 2.16445778]]), array([[-0.01948065,  1.17446528]])]
[[1.         1.        ]
 [1.84147098 1.84147098]
 [3.66006584 3.66006584]
 [4.08342586 4.08342586]
 [1.05621588 1.05621588]]
